In [97]:
import numpy as np
from collections import Counter
import Sudoku
import math
import random


def available_numbers(master, taken):
    master_arr = np.array([i for i in range(1, 10) for _ in range(9)])
    clue_arr = taken
    clueCount = Counter(clue_arr)
    fullCount = Counter(master_arr)
    remainCount = fullCount - clueCount
    result = np.array([val for val, cnt in remainCount.items() for _ in range(cnt)])
    return result
    

class SimulatedAnnealing:
    def __init__(self, sudoku_init, t_sched = None, t_init = 10):
        self.sudoku = sudoku_init
        self.grid = np.array(sudoku_init.board)
        self.fixed = np.array(sudoku_init.fixed)
        self.length = sudoku_init.length
        self.can_swap = ~self.fixed
        self.num_spots = self.can_swap.sum()
        self.possible_numbers = available_numbers(np.array([i for i in range(1, 10) for _ in range(9)]), self.grid.flatten())
        self.populate()
        
        self.T_init = t_init
        self.T = self.T_init
        self.iters = 1
        if t_sched==None:
            #self.decay = lambda n: n - 0.00002
            self.decay = 0.9999
        else:
            self.decay = t_sched
        self.error_count = 99999999
        self.T_min = 0.0000001
        
        #for testing
        self.num_goodswaps = 0
        self.num_badswaps = 0
        self.num_rejectedswaps=0
        
        
        
    def populate(self):
        for box_row in range(3):
            for box_col in range(3):
                numbers = set(range(1,10))
                for r in range(3*box_row, 3*box_row+3):
                    for c in range(3*box_col, 3*box_col+3):
                        if self.fixed[r][c]:
                            numbers.discard(self.grid[r][c])
                for r in range(3*box_row, 3*box_row+3):
                    for c in range(3*box_col, 3*box_col+3):
                        if not self.fixed[r][c]:
                            self.grid[r][c] = numbers.pop()
                        

    def swap_in_subgrid(self):
        box = random.randint(0, 8)
        row_start, col_start = 3 * (box // 3), 3 * (box % 3)
        candidates = [(r, c) for r in range(row_start, row_start + 3)
                            for c in range(col_start, col_start + 3)
                            if not self.fixed[r][c]]
        (r1, c1), (r2, c2) = random.sample(candidates, 2)
        self.grid[r1][c1], self.grid[r2][c2] = self.grid[r2][c2], self.grid[r1][c1]
        
    def swap(self):
        # --- 1. Choose a random 3×3 subgrid ---
        box = random.randint(0, 8)
        row_start, col_start = 3 * (box // 3), 3 * (box % 3)

        # --- 2. Collect all non-fixed positions in this subgrid ---
        candidates = [
            (r, c)
            for r in range(row_start, row_start + 3)
            for c in range(col_start, col_start + 3)
            if not self.fixed[r][c]
        ]

        # If fewer than two swappable cells exist, pick a new box
        if len(candidates) < 2:
            return self.swap()  # recursive retry

        # --- 3. Pick two random positions and swap them ---
        (r1, c1), (r2, c2) = random.sample(candidates, 2)
        self.grid[r1][c1], self.grid[r2][c2] = self.grid[r2][c2], self.grid[r1][c1]

        # --- 4. Compute error delta ---
        currErr = self.sudoku.numErrors()
        newErr = self.sudoku.numErrors(grid=self.grid.tolist())

        # --- 5. Acceptance rule ---
        delta = newErr - currErr
        if delta < 0:  # improvement
            self.sudoku.board = self.grid.tolist()
            self.error_count = newErr
            self.num_goodswaps += 1
        else:
            # probability of accepting worse move
            prob = math.exp(-delta / self.T)
            if random.random() < prob:
                self.sudoku.board = self.grid.tolist()
                self.error_count = newErr
                self.num_badswaps += 1
            else:
                # revert swap
                self.grid[r1][c1], self.grid[r2][c2] = self.grid[r2][c2], self.grid[r1][c1]
                self.num_rejectedswaps += 1

        # --- 6. Cool down ---
        self.iters += 1
        self.T *= self.decay

        
    def solve(self):
        while self.error_count>0:
            self.swap()
            if self.iters % 1000 ==0:
                print('Current Iteration: ', self.iters, '; Current errors: ', self.error_count, '; Current temperature: ', self.T)
                print('Good swaps: ', self.num_goodswaps, '; Bad swaps: ', self.num_badswaps, '; Rejected swaps: ', self.num_rejectedswaps)
                self.num_goodswaps = 0
                self.num_badswaps = 0
                self.num_rejectedswaps=0
            if self.iters % 20000 == 0:
                self.T *= 1.05  # small “reheat”
        print('final error count: ', self.error_count)
        print('final grid: ', self.grid)

                 

In [99]:
import Sudoku

board1 = Sudoku.Sudoku(3)
digitString = "070000043040009610800634900094052000358460020000800530080070091902100005007040802"
board1.fillFromString(digitString)
x = SimulatedAnnealing(board1)
x.solve()

Current Iteration:  1000 ; Current errors:  36 ; Current temperature:  9.049233858971451
Good swaps:  326 ; Bad swaps:  596 ; Rejected swaps:  77
Current Iteration:  2000 ; Current errors:  36 ; Current temperature:  8.188044457101192
Good swaps:  346 ; Bad swaps:  585 ; Rejected swaps:  69
Current Iteration:  3000 ; Current errors:  34 ; Current temperature:  7.408811958704974
Good swaps:  349 ; Bad swaps:  582 ; Rejected swaps:  69
Current Iteration:  4000 ; Current errors:  41 ; Current temperature:  6.7037367624262565
Good swaps:  342 ; Bad swaps:  573 ; Rejected swaps:  85
Current Iteration:  5000 ; Current errors:  34 ; Current temperature:  6.06576153240101
Good swaps:  344 ; Bad swaps:  551 ; Rejected swaps:  105
Current Iteration:  6000 ; Current errors:  36 ; Current temperature:  5.488500558998598
Good swaps:  325 ; Bad swaps:  575 ; Rejected swaps:  100
Current Iteration:  7000 ; Current errors:  31 ; Current temperature:  4.966175842096448
Good swaps:  309 ; Bad swaps:  55

In [13]:
import Sudoku

board1 = Sudoku.Sudoku(3)
digitString = "070000043040009610800634900094052000358460020000800530080070091902100005007040802"
board1.fillFromString(digitString)
x = SimulatedAnnealing(board1)
print(x.grid)
x.swap()
print(x.grid)


[[1 7 8 3 3 1 4 4 3]
 [5 4 8 1 5 9 6 1 3]
 [8 6 1 6 3 4 9 9 2]
 [6 9 4 1 5 2 7 1 5]
 [3 5 8 4 6 6 2 2 8]
 [8 5 9 8 5 6 5 3 9]
 [4 8 2 3 7 2 2 9 1]
 [9 3 2 1 7 4 6 9 5]
 [7 7 7 7 4 6 8 7 2]]
[[1 7 8 3 3 1 8 4 3]
 [5 4 4 1 5 9 6 1 3]
 [8 6 1 6 3 4 9 9 2]
 [6 9 4 1 5 2 7 1 5]
 [3 5 8 4 6 6 2 2 8]
 [8 5 9 8 5 6 5 3 9]
 [4 8 2 3 7 2 2 9 1]
 [9 3 2 1 7 4 6 9 5]
 [7 7 7 7 4 6 8 7 2]]


In [18]:
mask = np.array([True, False, True, True])
poss_nums = np.array([1,2,3])
expanded = np.zeros(mask.shape[0], dtype=int)
expanded[mask] = poss_nums[:mask.sum()]
print(expanded)

[1 0 2 3]
